In [25]:
import spacy
import re
from docx import Document
from spacy.matcher import Matcher
import json
import math

In [27]:
nlp = spacy.load("en_core_web_sm")
loaddoc=Document(r"Desktop/text.docx")
#loaddoc=Document(r"Desktop/Naṣīr al.docx")
text=[]
for paragraph in loaddoc.paragraphs:
    text.append(paragraph.text)
    #jointext='\n'.join(text)
    extracted='\n'.join(text)

doc=nlp(extracted) 
print(len(extracted))
print(type(extracted))

#print(len(tokenized))
#print(tokenized)

85255
<class 'str'>


In [29]:

lemma_nounlist=["birthday","born","die","publish","accomplishment","advisors","descendents","student","contribution","prize"]
lemma_verblist=["publish","influence","write","edit","work","contribution"]
matched=Matcher(nlp.vocab)

pattern1=[ {"POS":"PROPN","OP":"+"},{"POS":"VERB"},{"POS":"ADP"},{"IS_PUNCT":True,"OP": "?"}] 
pattern2=[ {"POS":"PRON","OP":"+"},{"POS":"VERB"},{"POS":"ADP"},{"IS_PUNCT":True,"OP": "?"}] 
pattern3=[{"POS":"NOUN","LEMMA":{"IN":lemma_nounlist}}] 
pattern4=[{"POS":"VERB","LEMMA":{"IN":lemma_verblist}}]

matched.add("PROPER_NOUN",[pattern1],greedy="LONGEST")
matched.add("PRONOUN",[pattern2],greedy="LONGEST")
matched.add("PROPER_NOUN",[pattern3],greedy="LONGEST")
matched.add("VERB",[pattern4],greedy="LONGEST")

matches=matched(doc)
matches.sort(key = lambda x:x[1])
print(len(matches))
lines=[]

for m in matches:
    #print(m, doc[m[1]:m[2]].sent.text)
    #print(doc[m[1]:m[2]].sent.text)
    if doc[m[1]:m[2]].sent.text not in lines:  
        #print(doc[m[1]:m[2]].sent.text)
        lines.append(doc[m[1]:m[2]].sent.text)
        extracted2='\n'.join(lines)
        
#print(extracted2)


clean=re.compile(r'https?://\S+|www\.\S+').sub(' ',extracted2) 
noref=re.compile(r'\[\d+\]').sub(' ',clean) 
extracted3=noref.replace('\\t',' ').replace('\n',' ').replace('\\xa0',' ').replace('\u00a0', '').replace('&nbsp;', '')

print(extracted3)
#print(type(extracted3))
doc2=nlp(extracted3)
#print(doc2)

108
Tusi was born in Tus in 1201 and died in Baghdad in 1274. Very little is known about his childhood and early education, apart from what he writes in his autobiography, Contemplation and Action (Sayr wa suluk). Later on he corresponded with Qaysari, the son-in-law of Ibn al-‘Arabi, and it seems that mysticism, as propagated by Sufi masters of his time, was not appealing to his mind ,and once the occasion was suitable, he composed his own manual of philosophical Sufism in the form of a small booklet entitled The Attributes of the Illustrious (Awsaf al-ashraf). The Mongol invasion and the turmoil it caused in the eastern Islamic territories hardly left the life of any of its citizens untouched. Although under Mongol domination, Tusi’s allegiance to any particular community or persuasion could not have been of any particular importance, the process itself paved the ground for Tusi to write on various aspects of Shi‘ism, both from Ismaili and Twelver Shi‘i viewpoints, with scholarly vig

In [9]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"   

nlp1 = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = [
    {'question': 'who is the biography about?','context': extracted3},
    {'question': 'what year he was born?','context': extracted3},
    {'question':'what year did he die?','context': extracted3},
    {'question':'which country was he born in?', 'context':extracted3},
    {'question': 'which country did he complete his education?','context':extracted3}
]
#extracted3=[]
res=nlp1(QA_input)
print(len(res))
summary={}
max_iter=5
#import json
for r in range(max_iter):
    if r==1:
        summary['name']=res[0]['answer']
        name=res[0]['answer']
    elif r==2:
        summary['birth']=res[1]['answer']
    elif r==3:
        summary['death']=res[2]['answer']
    elif r==4:
        summary['country-birth']=res[3]['answer']
    else:
        summary['country-education']=res[4]['answer']
    
json_pattern=json.dumps(summary)
print(json_pattern)

5
{"country-education": "Mosul", "name": "Nasir al-Din Tusi", "birth": "1201", "death": "1274", "country-birth": "Tus"}


In [17]:
tokenized=[]
for sent in doc2.sents:
    for token in sent:
        if token.lemma_ in ["compose", "write","publish","accomplishment"] and token.pos_ == "VERB":
            #start_idx = token.idx - sent.start_char
            #tokenized.append(sent.text[start_idx:])
            tokenized.append(sent.text)
            break     

if len(tokenized)!= len(set(tokenized)):
    print("Duplicates found")
else:
    print("No duplicate")
#print(tokenized)
result_string = '\n'.join(tokenized)
same=[]
model_name2 = "deepset/roberta-base-squad2" 
nlp2 = pipeline('question-answering', model=model_name2, tokenizer=model_name2,top_k=100)


words=result_string.split()
totalword=len(words)
word_persection=math.ceil(totalword/5)
sections=[]
for i in range(0,totalword,word_persection):
    sections.append(' '.join(words[i:i+word_persection]))


for i, section in enumerate(sections):
    print(f"Section {i+1}:")
    print(section)
    ques={'question':f'what did {name} write?','context':section}
    res4=nlp2(ques)
    same.append(res4)
    #print(res4)
    print()
#print(result_string)
combined=[]

for s in same:
    for inner in s:
        #print(inner['answer'])
        doc5=nlp(inner['answer']) #spacy
        for token in doc5.ents:
            #print(token.text, token.label_)
            if token.label_=='ORG' or token.label_=='WORK_OF_ART':
                if token.text not in combined:
                    combined.append(token.text)
       
print(len(combined))
print(combined)

summary2={}
summary2['written']=combined
json_pattern2=json.dumps(summary2)
print(json_pattern2)

No duplicate
Section 1:
Very little is known about his childhood and early education, apart from what he writes in his autobiography, Contemplation and Action (Sayr wa suluk). Later on he corresponded with Qaysari, the son-in-law of Ibn al-‘Arabi, and it seems that mysticism, as propagated by Sufi masters of his time, was not appealing to his mind ,and once the occasion was suitable, he composed his own manual of philosophical Sufism in the form of a small booklet entitled The Attributes of the Illustrious (Awsaf al-ashraf). Although under Mongol domination, Tusi’s allegiance to any particular community or persuasion could not have been of any particular importance, the process itself paved the ground for Tusi to write on various aspects of Shi‘ism, both from Ismaili and Twelver Shi‘i viewpoints, with scholarly vigour and enthusiasm. An encyclopedia of philosophy articles written by professional philosophers. They write new content and verify and edit content received from contributors